In [1]:
import os
dd=os.listdir('TIN')
f1 = open('train.txt', 'w')
f2 = open('test.txt', 'w')
for i in range(len(dd)):
    d2 = os.listdir ('TIN/%s/images/'%(dd[i]))
    for j in range(len(d2)-2):
        str1 = 'TIN/%s/images/%s'%(dd[i], d2[j])
        f1.write("%s %d\n" % (str1, i))
    str1 = 'TIN/%s/images/%s'%(dd[i], d2[-1])
    f2.write("%s %d\n" % (str1, i))
f1.close()
f2.close()

In [2]:
import numpy as np
from numpy import linalg as LA
from skimage.feature import hog
from sklearn.cluster import KMeans
import cv2

def load_img(f, func = "hog"):
    classes = list(range(10))
    f = open(f)
    lines = f.readlines()
    imgs, lab = [], []
    descriptors = []
    if func == "orb":
        orb = cv2.ORB_create()
    elif func == "brisk":
        brisk = cv2.BRISK_create()
    for i in range(len(lines)):
        line = lines[i].strip() # remove whitespaces
        filename, label = lines[i].split(' ')
        if int(label) in classes:
            im1 = cv2.imread(filename)
            im1 = cv2.resize(im1, (256,256))
            im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
            if func == "hog":
                des = hog(im1, orientations = 6, pixels_per_cell = (20, 20), cells_per_block = (2, 2)).reshape(-1, 1)
            elif func == "orb":
                key, des = orb.detectAndCompute(im1, None)
            elif func == "brisk":
                key, des = brisk.detectAndCompute(im1, None)
            if des is not None:
                descriptors.extend(des)
    descriptors = np.array(descriptors)
    # train a KMeans model for feature extraction to transform descriptor arrarys into arrarys with same size
    Kmeans_model = KMeans(n_clusters = len(classes) * 5, random_state = 2024).fit(descriptors)
    for i in range(len(lines)):
        line = lines[i].strip() # remove whitespace
        filename, label = lines[i].split(' ')
        if int(label) in classes:
            im1 = cv2.imread(filename)
            im1 = cv2.resize(im1, (256,256))
            im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
            if func == "hog":
                des = hog(im1, orientations = 6, pixels_per_cell = (20, 20), cells_per_block = (2, 2)).reshape(-1, 1)
            elif func == "orb":
                key, des = orb.detectAndCompute(im1, None)
            elif func == "brisk":
                key, des = brisk.detectAndCompute(im1, None)
            if des is not None:
                result = Kmeans_model.predict(des)
                vec = np.bincount(result, minlength = len(classes) * 5) # arrary filled with 0
        imgs.append(vec) 
        lab.append(int(label))
    imgs= np.asarray(imgs, np.float32)
    lab= np.asarray(lab, np.int32)
    return imgs, lab

In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
classifiers = {
        'K-Nearest Neighbor': KNeighborsClassifier(),
        'Support Vector Machine': SVC(),
        'CatBoost': CatBoostClassifier()
}
for name, model in classifiers.items():
    for func in ['hog','orb','brisk']:
        x, y = load_img('train.txt',func = func)
        tx, ty = load_img('test.txt', func = func)
        model.fit(x, y)
        pred = model.predict(tx)
        acc = accuracy_score(ty, pred)
        f1 = f1_score(ty, pred, average = "weighted")
        print(f"{name}({func}) - Accuracy: {acc}, F1-Score: {f1}")

c:\python38\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


K-Nearest Neighbor(hog) - Accuracy: 0.0, F1-Score: 0.0


c:\python38\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


K-Nearest Neighbor(orb) - Accuracy: 0.01, F1-Score: 0.008333333333333333


c:\python38\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


...

359:	learn: 5.0417639	total: 25m 36s	remaining: 45m 31s
360:	learn: 5.0416425	total: 25m 40s	remaining: 45m 26s
361:	learn: 5.0415812	total: 25m 44s	remaining: 45m 22s
362:	learn: 5.0414368	total: 25m 48s	remaining: 45m 18s
363:	learn: 5.0412978	total: 25m 53s	remaining: 45m 13s
364:	learn: 5.0411872	total: 25m 57s	remaining: 45m 9s
365:	learn: 5.0411337	total: 26m 1s	remaining: 45m 5s
366:	learn: 5.0410773	total: 26m 6s	remaining: 45m 1s
367:	learn: 5.0409255	total: 26m 10s	remaining: 44m 56s
368:	learn: 5.0408234	total: 26m 14s	remaining: 44m 52s
369:	learn: 5.0407739	total: 26m 18s	remaining: 44m 48s
370:	learn: 5.0407505	total: 26m 22s	remaining: 44m 43s
371:	learn: 5.0406084	total: 26m 27s	remaining: 44m 39s
372:	learn: 5.0405917	total: 26m 31s	remaining: 44m 35s
373:	learn: 5.0405423	total: 26m 35s	remaining: 44m 31s
374:	learn: 5.0404187	total: 26m 40s	remaining: 44m 26s
375:	learn: 5.0403120	total: 26m 44s	remaining: 44m 22s
376:	learn: 5.0402525	total: 26m 48s	remaining: 44m 1

507:	learn: 5.0294930	total: 36m 5s	remaining: 34m 57s
508:	learn: 5.0294021	total: 36m 9s	remaining: 34m 52s
509:	learn: 5.0292885	total: 36m 13s	remaining: 34m 48s
510:	learn: 5.0292248	total: 36m 18s	remaining: 34m 44s
511:	learn: 5.0291192	total: 36m 22s	remaining: 34m 40s
512:	learn: 5.0290291	total: 36m 26s	remaining: 34m 35s
513:	learn: 5.0289395	total: 36m 30s	remaining: 34m 31s
514:	learn: 5.0288635	total: 36m 35s	remaining: 34m 27s
515:	learn: 5.0287938	total: 36m 39s	remaining: 34m 22s
516:	learn: 5.0286813	total: 36m 43s	remaining: 34m 18s
517:	learn: 5.0285989	total: 36m 47s	remaining: 34m 14s
518:	learn: 5.0285406	total: 36m 51s	remaining: 34m 9s
519:	learn: 5.0285200	total: 36m 56s	remaining: 34m 5s
520:	learn: 5.0284588	total: 37m	remaining: 34m 1s
521:	learn: 5.0284225	total: 37m 4s	remaining: 33m 57s
522:	learn: 5.0283167	total: 37m 8s	remaining: 33m 52s
523:	learn: 5.0282634	total: 37m 13s	remaining: 33m 48s
524:	learn: 5.0282096	total: 37m 17s	remaining: 33m 44s
525

655:	learn: 5.0204519	total: 46m 26s	remaining: 24m 21s
656:	learn: 5.0204256	total: 46m 30s	remaining: 24m 17s
657:	learn: 5.0203585	total: 46m 34s	remaining: 24m 12s
658:	learn: 5.0203384	total: 46m 39s	remaining: 24m 8s
659:	learn: 5.0202681	total: 46m 43s	remaining: 24m 4s
660:	learn: 5.0202548	total: 46m 47s	remaining: 23m 59s
661:	learn: 5.0202075	total: 46m 51s	remaining: 23m 55s
662:	learn: 5.0201175	total: 46m 55s	remaining: 23m 51s
663:	learn: 5.0200879	total: 47m	remaining: 23m 47s
664:	learn: 5.0200183	total: 47m 4s	remaining: 23m 42s
665:	learn: 5.0199575	total: 47m 8s	remaining: 23m 38s
666:	learn: 5.0199170	total: 47m 12s	remaining: 23m 34s
667:	learn: 5.0197923	total: 47m 16s	remaining: 23m 29s
668:	learn: 5.0197208	total: 47m 21s	remaining: 23m 25s
669:	learn: 5.0196926	total: 47m 25s	remaining: 23m 21s
670:	learn: 5.0195947	total: 47m 29s	remaining: 23m 17s
671:	learn: 5.0195484	total: 47m 33s	remaining: 23m 12s
672:	learn: 5.0194849	total: 47m 37s	remaining: 23m 8s
6

803:	learn: 5.0130624	total: 56m 46s	remaining: 13m 50s
804:	learn: 5.0130171	total: 56m 50s	remaining: 13m 46s
805:	learn: 5.0129509	total: 56m 54s	remaining: 13m 41s
806:	learn: 5.0129206	total: 56m 58s	remaining: 13m 37s
807:	learn: 5.0128414	total: 57m 3s	remaining: 13m 33s
808:	learn: 5.0127960	total: 57m 7s	remaining: 13m 29s
809:	learn: 5.0127886	total: 57m 11s	remaining: 13m 24s
810:	learn: 5.0127688	total: 57m 15s	remaining: 13m 20s
811:	learn: 5.0127395	total: 57m 19s	remaining: 13m 16s
812:	learn: 5.0126889	total: 57m 24s	remaining: 13m 12s
813:	learn: 5.0126376	total: 57m 28s	remaining: 13m 7s
814:	learn: 5.0125770	total: 57m 32s	remaining: 13m 3s
815:	learn: 5.0125564	total: 57m 36s	remaining: 12m 59s
816:	learn: 5.0125333	total: 57m 40s	remaining: 12m 55s
817:	learn: 5.0124714	total: 57m 45s	remaining: 12m 50s
818:	learn: 5.0124573	total: 57m 49s	remaining: 12m 46s
819:	learn: 5.0124495	total: 57m 53s	remaining: 12m 42s
820:	learn: 5.0123950	total: 57m 57s	remaining: 12m 

950:	learn: 5.0077451	total: 1h 7m 4s	remaining: 3m 27s
951:	learn: 5.0076911	total: 1h 7m 8s	remaining: 3m 23s
952:	learn: 5.0076797	total: 1h 7m 13s	remaining: 3m 18s
953:	learn: 5.0076693	total: 1h 7m 17s	remaining: 3m 14s
954:	learn: 5.0076389	total: 1h 7m 21s	remaining: 3m 10s
955:	learn: 5.0076168	total: 1h 7m 25s	remaining: 3m 6s
956:	learn: 5.0075885	total: 1h 7m 30s	remaining: 3m 1s
957:	learn: 5.0075543	total: 1h 7m 34s	remaining: 2m 57s
958:	learn: 5.0075040	total: 1h 7m 38s	remaining: 2m 53s
959:	learn: 5.0074918	total: 1h 7m 42s	remaining: 2m 49s
960:	learn: 5.0074767	total: 1h 7m 46s	remaining: 2m 45s
961:	learn: 5.0074342	total: 1h 7m 51s	remaining: 2m 40s
962:	learn: 5.0073972	total: 1h 7m 55s	remaining: 2m 36s
963:	learn: 5.0073937	total: 1h 7m 59s	remaining: 2m 32s
964:	learn: 5.0073473	total: 1h 8m 3s	remaining: 2m 28s
965:	learn: 5.0073239	total: 1h 8m 7s	remaining: 2m 23s
966:	learn: 5.0072768	total: 1h 8m 12s	remaining: 2m 19s
967:	learn: 5.0072177	total: 1h 8m 16